# ZK Auth Prototype — Threat Model + Protocol Walkthrough

This notebook is a portfolio-grade *systems design* artefact for a ZK authentication flow.

It includes:
- adversary/threat model
- protocol sketch (registration + login)
- security properties + failure cases
- minimal toy implementation of a challenge-response proof (non-cryptographic placeholder)

Note: This notebook does **not** implement real ZK proofs. It documents architecture and provides a safe toy demo.
Outputs are saved in the notebook when executed.

## Threat model (high-level)
**Attacker goals**:
- credential stuffing / replay
- phishing and session hijack
- database exfiltration (verifier secrets)

**Assumptions**:
- TLS between client and server
- server stores no raw password

**Desired properties**:
- no password-equivalent leaks
- replay-resistant proofs (nonce)
- device binding optional
- rate limiting + abuse controls

In [1]:
import hashlib
import os
import hmac
import secrets
from dataclasses import dataclass

def H(x: bytes) -> bytes:
    return hashlib.sha256(x).digest()

def hx(b: bytes) -> str:
    return b.hex()

## Toy protocol (NOT ZK): nonce-bound proof of possession
We model a client secret `s` and a stored verifier `v = H(s)`.

Login proof: client returns `p = HMAC(s, nonce)` and server verifies using `v` is not possible directly.
So we instead store server-side a derived key `k = H(s || salt)` (this becomes password-equivalent, so not ZK).

This is a **toy** to illustrate why we want ZK: so the server can verify without storing password-equivalent material.

In [2]:
@dataclass
class UserRecord:
    salt: bytes
    key: bytes

def register(secret: str) -> UserRecord:
    salt = os.urandom(16)
    key = H(secret.encode('utf-8') + salt)
    return UserRecord(salt=salt, key=key)

def prove(secret: str, nonce: bytes, record: UserRecord) -> bytes:
    key = H(secret.encode('utf-8') + record.salt)
    return hmac.new(key, nonce, hashlib.sha256).digest()

def verify(proof: bytes, nonce: bytes, record: UserRecord) -> bool:
    expected = hmac.new(record.key, nonce, hashlib.sha256).digest()
    return hmac.compare_digest(proof, expected)

record = register('correct horse battery staple')
nonce = secrets.token_bytes(16)
proof_ok = prove('correct horse battery staple', nonce, record)
proof_bad = prove('wrong password', nonce, record)
verify(proof_ok, nonce, record), verify(proof_bad, nonce, record)

(True, False)

## Takeaway
The toy scheme demonstrates replay resistance via nonce, but it is **not ZK** because the server stores a password-equivalent key.

Next steps for real ZK:
- pick a construction (OPAQUE / SRP-like / zk-SNARK-based)
- formalize statements and transcripts
- implement in circuits (Circom) + verify on server